In [1]:
#### import tatqa_utils
import pandas as pd
import table_convert
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2


In [2]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [3]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [4]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [42]:
from openai import OpenAI

def gen_code2(question, value_list):
 
    client = OpenAI()
    
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
         {
          "role": "system",
          "content": "You are a helpful assistant with a subtask in a question-answering pipeline. The questions are related to a financial report. " + 
             "Financial report is stored as a list of annotated values. You will receive the financial report as an annotated value list and the question. "+
             "Your task is to generate a Python function that can calculate a numeric value that is the answer for the received question."
             
            # "Here is an example:\n\n"+
            # "Question: \nWhat is the ratio of IMFT’s total assets to total liabilities in 2019? \n" + 
            # "Value list: \n[{'number_value': 118.0, 'scale': '', 'category': 'Accounts payable and accrued expenses', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 138.0, 'scale': '', 'category': 'Accounts payable and accrued expenses', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 696.0, 'scale': '', 'category': 'Current debt', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 20.0, 'scale': '', 'category': 'Current debt', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 37.0, 'scale': '', 'category': 'Other current liabilities', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 9.0, 'scale': '', 'category': 'Other current liabilities', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 851.0, 'scale': '', 'category': 'Total current liabilities', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 167.0, 'scale': '', 'category': 'Total current liabilities', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 53.0, 'scale': '', 'category': 'Long-term debt', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 1064.0, 'scale': '', 'category': 'Long-term debt', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 5.0, 'scale': '', 'category': 'Other noncurrent liabilities', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 74.0, 'scale': '', 'category': 'Other noncurrent liabilities', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 909.0, 'scale': '', 'category': 'Total liabilities', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 1305.0, 'scale': '', 'category': 'Total liabilities', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}] \n"+
            # "Generated code: \n"+
            #"""
            #def run(value_list):
            #    total_assets = 0
            #    total_liabilities = 0
                
            #    for item in value_list:
            #        if item['category'] == 'Total assets':
            #            total_assets = item['number_value']
            #        elif item['category'] == 'Total liabilities':
            #            total_liabilities = item['number_value']
                
            #    ratio = total_assets / total_liabilities
            #    return (round(ratio, 2), '')"
            #""" 
        },
        {
          "role": "user",
          "content": f"Here is the financial report as a list of annotated values: {value_list}"
        },         
        {
          "role": "assistant",
          "content": "Ok, I received the value list."
        },
        {
          "role": "user",
          "content": f"Here is the question requires calculation on the financial report: {question}"
        },
        {
          "role": "assistant",
          "content": "Ok, I received the question."
        },
        {
          "role": "user",
          "content": "Generate a Python function 'run(value_list)' that can answer the question using the list of annotated values! "+
            "The function must return a tuple (number, scale). The resulting number is a float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. "+
            "The calculation usually involves two steps: a selection and a calculation on selected data. "+
            "If the question is about calculating the average value in a given year, you have to calculate the average between the given year and the previous one. It is aka Average Book Value. A value always represents year-end value.  "+
            "Do not generate explanation, nor example code, just the function."
        },
        {
          "role": "assistant",
          "content": "Ok, I have all the information. The Python function is as follows:"
        },
      ],
      temperature=0,
      #max_tokens=64,
      top_p=1
    )
    code =  response.choices[0].message.content.replace('```python','').replace('```','')
    #code = response.choises[0].message.content.replace('```python','').replace('```','')
    #return response
    #code = response.content.replace('```python','').replace('```','')
    return ("", code)
    

def gen_code(llm, question, value_list):  
        #prompt = f"Generate a Python function 'run(table)' that can answer the following question using the following table! The function must return numeric float results with accuracy to two decimal places. If result value or calculation is percent calculation return format (value, 'percent'), every orher case return (value, '') . Do not generate explanation, nor example code, just the function.\n\nQuestion: {question}  \n\nTable: {table}"
        prompt = f"You are a helpful assistant to answer questions in the financial reports domain. Generate a Python function 'run(value_list)' that can answer the following question using the following JSON value list, annotated with category and metadata! The function must return a tuple (number, scale). Result number is float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. Always use categories to select the proper values for calculation. Usually year selection is necessary the proper values for calculation. Do not generate explanation, nor example code, just the function. \n\nQuestion: {question}  \n\Value list: {value_list}"
        res = llm.invoke(prompt)
        code = res.content.replace('```python','').replace('```','')
        return (prompt, code)
    
def exec_code(code, value_list):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({value_list})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')


In [43]:
vl =  [{'number_value': 118.0, 'scale': '', 'category': 'Accounts payable and accrued expenses', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 138.0, 'scale': '', 'category': 'Accounts payable and accrued expenses', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 696.0, 'scale': '', 'category': 'Current debt', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 20.0, 'scale': '', 'category': 'Current debt', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 37.0, 'scale': '', 'category': 'Other current liabilities', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 9.0, 'scale': '', 'category': 'Other current liabilities', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 851.0, 'scale': '', 'category': 'Total current liabilities', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 167.0, 'scale': '', 'category': 'Total current liabilities', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 53.0, 'scale': '', 'category': 'Long-term debt', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 1064.0, 'scale': '', 'category': 'Long-term debt', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 5.0, 'scale': '', 'category': 'Other noncurrent liabilities', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 74.0, 'scale': '', 'category': 'Other noncurrent liabilities', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}, {'number_value': 909.0, 'scale': '', 'category': 'Total liabilities', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}, {'number_value': 1305.0, 'scale': '', 'category': 'Total liabilities', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}]
def run(value_list):
    total_assets = 0
    total_liabilities = 0
    
    for item in value_list:
        if item['category'] == 'Total assets':
            total_assets = item['number_value']
            print(total_assets)
        elif item['category'] == 'Total liabilities':
            total_liabilities = item['number_value']
            print(total_liabilities)
    ratio = total_assets / total_liabilities
    
    return (round(ratio, 2), '')
run(vl)


909.0
1305.0


(0.0, '')

In [44]:
1305.0 /909.0

1.4356435643564356

In [45]:
[print(l) for l in vl]

{'number_value': 118.0, 'scale': '', 'category': 'Accounts payable and accrued expenses', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}
{'number_value': 138.0, 'scale': '', 'category': 'Accounts payable and accrued expenses', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header6': '8', 'header7': '114', 'header8': '126', 'header9': '$91', 'header10': '', 'header11': '2018'}
{'number_value': 696.0, 'scale': '', 'category': 'Current debt', 'header1': '', 'header2': '$2,664', 'header3': '38', 'header4': '2,235', 'header5': '391', 'header6': '9', 'header7': '124', 'header8': '128', 'header9': '$130', 'header10': '', 'header11': '2019'}
{'number_value': 20.0, 'scale': '', 'category': 'Current debt', 'header1': '', 'header2': '$3,025', 'header3': '45', 'header4': '2,641', 'header5': '339', 'header

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [46]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')
item = devdf.iloc[0]
table = item['table']['table']
#table = table_convert.fill_table_headers(table)
value_list = table_convert.convert_multitable(table)


pars = [p['text'] for p in item['paragraphs']]
question = item['questions'][5]['question']
print(question)
#( prompt, code) = gen_code(llm, question, value_list)
( prompt, code) = gen_code2(question, value_list)
res = exec_code(code, value_list)
print(value_list, code, res,  item['questions'][5]['answer'])

What is the percentage change in Other in 2019 from 2018?
[{'number_value': 1452.4, 'scale': '', 'category': 'Fixed Price', 'header1': '2019', 'header2': 'Years Ended September 30,'}, {'number_value': 1146.2, 'scale': '', 'category': 'Fixed Price', 'header1': '2018', 'header2': 'Years Ended September 30,'}, {'number_value': 1036.9, 'scale': '', 'category': 'Fixed Price', 'header1': '2017', 'header2': 'Years Ended September 30,'}, {'number_value': 44.1, 'scale': '', 'category': 'Other', 'header1': '2019', 'header2': 'Years Ended September 30,'}, {'number_value': 56.7, 'scale': '', 'category': 'Other', 'header1': '2018', 'header2': 'Years Ended September 30,'}, {'number_value': 70.8, 'scale': '', 'category': 'Other', 'header1': '2017', 'header2': 'Years Ended September 30,'}, {'number_value': 1496.5, 'scale': '', 'category': 'Total sales', 'header1': '2019', 'header2': 'Years Ended September 30,'}, {'number_value': 1202.9, 'scale': '', 'category': 'Total sales', 'header1': '2018', 'heade

In [47]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    value_list = table_convert.convert_multitable(table)
    
    #docs = valuelist2docs(value_list)
    #v_vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())
    #v_retriever = v_vectorstore.as_retriever()
            
    #pars = [p['text'] for p in item['paragraphs']]
    #table = transform_elements(table, remove_thousand_separators)
    #table = [remove_thousand_separators(cell) for row in table for cell in row]
    #table = [cell.replace('$', '') for row in table for cell in row]
    #print (table)
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']:            
            #sel = v_retriever.invoke(question)
            #sel_txt = '[' + ','.join([i.page_content for i in sel])  + ']'

            #( prompt, code)  = gen_code(llm, q['question'], value_list)
            ( prompt, code)  = gen_code2(q['question'], value_list)
            r = exec_code(code, value_list)
            err =  None
            if r is None:
                (pred_value, pred_scale) = ("", "")
            else:   
                if len(r) == 2:                    
                    (pred_value, pred_scale) = r    
                    if isinstance(pred_value, tuple) and len(pred_value) == 2:
                        print('$$$$')
                        (pred_value, pred_scale) = pred_value                     
                    if pred_scale == "%" or pred_scale == "percentage"  :
                        pred_scale = 'percent'
                    if pred_scale not in ["", 'thousand', 'million', 'billion', 'percent']:
                        print('Invalid ', pred_scale)
                        pred_scale = ""    
                    #if item['table']['uid'] in res_scale and pred_scale != 'percent':
                    #    pred_scale = res_scale[item['table']['uid']]
                    if isinstance(pred_value, str):
                        #print("string")
                        if  pred_value.startswith('[Error]'):
                            (pred_value, pred_scale) = ("", "")
                            err,_ = r                                                        
                        
                elif len(r) == 1:
                    (pred_value, pred_scale) = (r, "")        
                else:
                    (pred_value, pred_scale) = ("", "")        
                
                
                
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_scale, q, code, prompt,value_list, err))



0it [00:00, ?it/s]

[Error]
[Error]
Invalid  $’000
Invalid  $’000
Invalid  £m
[Error]unsupported operand type(s) for -: 'float' and 'NoneType'
[Error]unsupported operand type(s) for -: 'NoneType' and 'NoneType'
[Error]division by zero


IndexError: list index out of range

In [16]:
table

[['', '', 'As of February 28, 2019', ''],
 ['', '', 'ASC 606', 'Without ASC 606'],
 ['', 'As reported', 'Adjustments', 'Adoption'],
 ['Assets', '', '', ''],
 ['Prepaid expenses and other current assets (1)',
  '$19,373',
  '(1,473)',
  '$17,900'],
 ['Deferred income tax assets', '22,626', '(532)', '22,094'],
 ['Other assets (1)', '22,510', '(3,319)', '19,191'],
 ['', "Liabilities and Stockholders' Equity", '', ''],
 ['Deferred revenue (2)', '$24,264', '(1,945)', '22,319'],
 ['Other non-current liabilities (2)', '38,476', '(5,353)', '33,123'],
 ["Stockholders' equity:", '', '', ''],
 ['Accumulated deficit', '$(2,227)', '1,689', '(538)']]

In [48]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=False)
print( pred_em, pred_f1, scale_score)

0.5555555555555556 0.5555555555555556 0.6111111111111112


In [49]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    
    if pred  is None or isinstance(pred, tuple) or isinstance(pred, str):
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #type error
        #if isinstance(pred, str):
        #    print (pred, pred_scale, ' -- ' , ans['answer'])
        #if isinstance(pred, tuple):
        #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        #print('===============================================================================')
        print('{',pred, '}', ans['answer'], ans['scale'], err)
        continue
    
    #pred = round(pred, 2)
    
    if (pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit:
        good = good + 1
        #if ans['scale'] == pred_scale and  pred_scale == 'thousand':
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' == ' , ans['answer'], ans['scale'])
        
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if  ans['scale'] == 'thousand':
        #    print (q['uid'], pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
          
    else:
        #value error
        #  if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if ans['answer'] != pred and ans['scale'] == pred_scale and pred_scale != "":
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #None
        print (q['uid'],q['derivation'], pred, pred_scale, ' <> ' , ans['answer'], ans['scale'])
        print(q['question'])
        print(value_list)
        #print(value_list)
        print(code)
        
    #if isinstance(pred, str):
    #    print (pred, pred_scale, ' -- ' , ans['answer'])
    #if isinstance(pred, tuple):
    #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        
print (good/len(res))

263d03ec-83d2-48df-8376-1a72167798f7 2,641/3,025  87.31 percent  <>  0.87 
What is the proportion of IMFT’s property, plant, and equipment over total assets in 2018?
[{'number_value': 130.0, 'scale': '', 'category': 'Cash and equivalents', 'header1': '', 'header2': '2019'}, {'number_value': 91.0, 'scale': '', 'category': 'Cash and equivalents', 'header1': '', 'header2': '2018'}, {'number_value': 128.0, 'scale': '', 'category': 'Receivables', 'header1': '', 'header2': '2019'}, {'number_value': 126.0, 'scale': '', 'category': 'Receivables', 'header1': '', 'header2': '2018'}, {'number_value': 124.0, 'scale': '', 'category': 'Inventories', 'header1': '', 'header2': '2019'}, {'number_value': 114.0, 'scale': '', 'category': 'Inventories', 'header1': '', 'header2': '2018'}, {'number_value': 9.0, 'scale': '', 'category': 'Other current assets', 'header1': '', 'header2': '2019'}, {'number_value': 8.0, 'scale': '', 'category': 'Other current assets', 'header1': '', 'header2': '2018'}, {'number_v

In [13]:
len(res)

36

In [168]:
value_list = [{'number_value': 4.0, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2019', 'September 30,', 'Domestic']}, {'number_value': 3.75, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2018', 'September 30,', 'Domestic']}, {'number_value': 1.9, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2019', 'September 30,', 'International']}, {'number_value': 2.8, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2018', 'September 30,', 'International']}, {'number_value': 3.4, 'scale': '%', 'categories': ['Expected return on plan assets'], 'metadata': ['2019', 'September 30,', 'International']}, {'number_value': 3.7, 'scale': '%', 'categories': ['Expected return on plan assets'], 'metadata': ['2018', 'September 30,', 'International']}]

def run(value_list):
    domestic_2019 = None
    domestic_2018 = None
    international_2019 = None
    international_2018 = None
    
    for value in value_list:
        if 'Discount rate' in value['categories']:
            if '2019' in value['metadata'] and 'Domestic' in value['metadata']:
                domestic_2019 = value['number_value']
            elif '2018' in value['metadata'] and 'Domestic' in value['metadata']:
                domestic_2018 = value['number_value']
            elif '2019' in value['metadata'] and 'International' in value['metadata']:
                international_2019 = value['number_value']
            elif '2018' in value['metadata'] and 'International' in value['metadata']:
                international_2018 = value['number_value']
    print()
    domestic_diff = abs(domestic_2019 - domestic_2018)
    international_diff = abs(international_2019 - international_2018)
    
    return (round(abs(domestic_diff - international_diff), 2), '')

run(value_list)


(0.65, '')

In [12]:
wrong_indexes = [idx for idx, m in enumerate(metrics._details) if m['f1']<0.2]
resdf = pd.DataFrame(res)
resdf2 = resdf[resdf.index.isin(wrong_indexes)]

resdf2["answer"] = resdf2[0].apply(lambda x: x['answer'])
resdf2["scale"] = resdf2[0].apply(lambda x: x['scale'])
resdf2["qid"] = resdf2[3].apply(lambda x: x['uid'])
resdf2["q"] = resdf2[3].apply(lambda x: x['question'])
resdf2

/tmp/ipykernel_770237/596054401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resdf2["answer"] = resdf2[0].apply(lambda x: x['answer'])
/tmp/ipykernel_770237/596054401.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resdf2["scale"] = resdf2[0].apply(lambda x: x['scale'])
/tmp/ipykernel_770237/596054401.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

,0,1,2,3,4,5,6,7,answer,scale,qid,q
0,"{'answer_type': 'arithmetic', 'answer': -12.6,...",-12.6,,{'uid': 'eb787966-fa02-401f-bfaf-ccabf3828b23'...,\ndef run(value_list):\n other_2019 = next(...,,"[{'number_value': 1452.4, 'scale': '', 'catego...",None,-12.60,million,eb787966-fa02-401f-bfaf-ccabf3828b23,What is the change in Other in 2019 from 2018?
7,"{'answer_type': 'arithmetic', 'answer': 172, '...",166.0,million,{'uid': 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'...,\ndef run(value_list):\n total = 0\n cou...,,"[{'number_value': 166.0, 'scale': 'million', '...",None,172.00,million,a0414f81-8dc2-44b2-a441-2c9d9c805c4d,What is the 2019 average defined contribution ...
8,"{'answer_type': 'arithmetic', 'answer': 50.5, ...",57.0,million,{'uid': 'bf7abd62-d9cd-48d2-8826-1457684019a3'...,\ndef run(value_list):\n total = 0\n cou...,,"[{'number_value': 166.0, 'scale': 'million', '...",None,50.50,million,bf7abd62-d9cd-48d2-8826-1457684019a3,What is the 2019 average defined benefit schemes?
9,"{'answer_type': 'arithmetic', 'answer': 121.5,...",138.33,million,{'uid': '4d259081-6da6-44bd-8830-e4de0031744c'...,\ndef run(value_list):\n dc_values = [item[...,,"[{'number_value': 166.0, 'scale': 'million', '...",None,121.50,million,4d259081-6da6-44bd-8830-e4de0031744c,What is the difference between 2019 average de...
13,"{'answer_type': 'arithmetic', 'answer': 2.93, ...",0.0,,{'uid': 'ba6783f3-8207-419a-b407-3f688682caef'...,\ndef run(value_list):\n total_assets = 0\n...,,"[{'number_value': 118.0, 'scale': '', 'categor...",None,2.93,,ba6783f3-8207-419a-b407-3f688682caef,What is the ratio of IMFT’s total assets to to...
14,"{'answer_type': 'arithmetic', 'answer': 0.87, ...",81.53,percent,{'uid': '263d03ec-83d2-48df-8376-1a72167798f7'...,\ndef run(value_list):\n total_assets_2018 ...,,"[{'number_value': 118.0, 'scale': '', 'categor...",None,0.87,,263d03ec-83d2-48df-8376-1a72167798f7,"What is the proportion of IMFT’s property, pla..."
15,"{'answer_type': 'arithmetic', 'answer': -361, ...",-396.0,,{'uid': '78fc6d55-c20c-4f71-99fe-bc40a16e61d0'...,\ndef run(value_list):\n total_assets_2018 ...,,"[{'number_value': 118.0, 'scale': '', 'categor...",None,-361.00,,78fc6d55-c20c-4f71-99fe-bc40a16e61d0,What is the change of IMFT’s total assets from...
16,"{'answer_type': 'arithmetic', 'answer': 73, 's...",,,{'uid': '35d602ae-9131-4291-a30c-49a40f32bbe4'...,\ndef run(value_list):\n vat_receivables_20...,,[],[Error],73.00,thousand,35d602ae-9131-4291-a30c-49a40f32bbe4,What was the change in Value added tax receiva...
17,"{'answer_type': 'arithmetic', 'answer': 14.07,...",,,{'uid': 'a4f782f1-de3b-47a3-87af-1cb33c112b39'...,\ndef run(value_list):\n vat_receivables_20...,,[],[Error],14.07,percent,a4f782f1-de3b-47a3-87af-1cb33c112b39,What was the percentage change in Value added ...
18,"{'answer_type': 'arithmetic', 'answer': 4227.5...",4411.0,million,{'uid': 'dc5e217a-a7b3-4fc9-ac0f-13d328f26b20'...,\ndef run(value_list):\n total_free_cash_fl...,,"[{'number_value': 14182.0, 'scale': 'million',...",None,4227.50,million,dc5e217a-a7b3-4fc9-ac0f-13d328f26b20,What is the 2019 average free cash flow?


In [13]:
resdf2

,0,1,2,3,4,5,6,7,answer,scale,qid,q
0,"{'answer_type': 'arithmetic', 'answer': -12.6,...",-12.6,,{'uid': 'eb787966-fa02-401f-bfaf-ccabf3828b23'...,\ndef run(value_list):\n other_2019 = next(...,,"[{'number_value': 1452.4, 'scale': '', 'catego...",None,-12.60,million,eb787966-fa02-401f-bfaf-ccabf3828b23,What is the change in Other in 2019 from 2018?
7,"{'answer_type': 'arithmetic', 'answer': 172, '...",166.0,million,{'uid': 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'...,\ndef run(value_list):\n total = 0\n cou...,,"[{'number_value': 166.0, 'scale': 'million', '...",None,172.00,million,a0414f81-8dc2-44b2-a441-2c9d9c805c4d,What is the 2019 average defined contribution ...
8,"{'answer_type': 'arithmetic', 'answer': 50.5, ...",57.0,million,{'uid': 'bf7abd62-d9cd-48d2-8826-1457684019a3'...,\ndef run(value_list):\n total = 0\n cou...,,"[{'number_value': 166.0, 'scale': 'million', '...",None,50.50,million,bf7abd62-d9cd-48d2-8826-1457684019a3,What is the 2019 average defined benefit schemes?
9,"{'answer_type': 'arithmetic', 'answer': 121.5,...",138.33,million,{'uid': '4d259081-6da6-44bd-8830-e4de0031744c'...,\ndef run(value_list):\n dc_values = [item[...,,"[{'number_value': 166.0, 'scale': 'million', '...",None,121.50,million,4d259081-6da6-44bd-8830-e4de0031744c,What is the difference between 2019 average de...
13,"{'answer_type': 'arithmetic', 'answer': 2.93, ...",0.0,,{'uid': 'ba6783f3-8207-419a-b407-3f688682caef'...,\ndef run(value_list):\n total_assets = 0\n...,,"[{'number_value': 118.0, 'scale': '', 'categor...",None,2.93,,ba6783f3-8207-419a-b407-3f688682caef,What is the ratio of IMFT’s total assets to to...
14,"{'answer_type': 'arithmetic', 'answer': 0.87, ...",81.53,percent,{'uid': '263d03ec-83d2-48df-8376-1a72167798f7'...,\ndef run(value_list):\n total_assets_2018 ...,,"[{'number_value': 118.0, 'scale': '', 'categor...",None,0.87,,263d03ec-83d2-48df-8376-1a72167798f7,"What is the proportion of IMFT’s property, pla..."
15,"{'answer_type': 'arithmetic', 'answer': -361, ...",-396.0,,{'uid': '78fc6d55-c20c-4f71-99fe-bc40a16e61d0'...,\ndef run(value_list):\n total_assets_2018 ...,,"[{'number_value': 118.0, 'scale': '', 'categor...",None,-361.00,,78fc6d55-c20c-4f71-99fe-bc40a16e61d0,What is the change of IMFT’s total assets from...
16,"{'answer_type': 'arithmetic', 'answer': 73, 's...",,,{'uid': '35d602ae-9131-4291-a30c-49a40f32bbe4'...,\ndef run(value_list):\n vat_receivables_20...,,[],[Error],73.00,thousand,35d602ae-9131-4291-a30c-49a40f32bbe4,What was the change in Value added tax receiva...
17,"{'answer_type': 'arithmetic', 'answer': 14.07,...",,,{'uid': 'a4f782f1-de3b-47a3-87af-1cb33c112b39'...,\ndef run(value_list):\n vat_receivables_20...,,[],[Error],14.07,percent,a4f782f1-de3b-47a3-87af-1cb33c112b39,What was the percentage change in Value added ...
18,"{'answer_type': 'arithmetic', 'answer': 4227.5...",4411.0,million,{'uid': 'dc5e217a-a7b3-4fc9-ac0f-13d328f26b20'...,\ndef run(value_list):\n total_free_cash_fl...,,"[{'number_value': 14182.0, 'scale': 'million',...",None,4227.50,million,dc5e217a-a7b3-4fc9-ac0f-13d328f26b20,What is the 2019 average free cash flow?
